In [ ]:
!python -m  pip install tflite-runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.7 MB/s eta 0:00:00


In [ ]:
######################################
# 1-1. 기본: 텐서플로우 라이트로 변환하기
####################################

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten,Dense
from tensorflow.keras.optimizers import Adam
import pathlib

myLayer = Dense(units=1, input_shape=[1])
model = Sequential([myLayer])
model.compile(
    optimizer='sgd',
    loss = 'mean_squared_error'
)

xs = np.array([-1.0,0.0,1.0,2.0,3.0,4.0],dtype=float)
ys = np.array([-3.0,-1.0,1.0,3.0,5.0,7.0],dtype = float)

model.fit(xs,ys,epochs=500, verbose=0)

print(model.predict([10.0]))
print("학습된 가중치: {}".format(myLayer.get_weights()))
print("get Config: {}".format(myLayer.get_config()))

#모델 저장하기
export_dir='/content/drive/MyDrive/INTEL_PYTHON/saved_model/1' #디렉토리 생성해줌
tf.saved_model.save(model,export_dir)


#모델 변환하기
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert() #tflite 모델로 저장하기

#.tflite 모델로 저장하기
tflite_model_file = pathlib.Path('/content/drive/MyDrive/INTEL_PYTHON/model.tflite')
tflite_model_file.write_bytes(tflite_model)





Epoch 1/500
1/1 [==============================] - 0s 296ms/step - loss: 3.6283
Epoch 2/500
1/1 [==============================] - 0s 12ms/step - loss: 3.0169
Epoch 3/500
1/1 [==============================] - 0s 9ms/step - loss: 2.5325
Epoch 4/500
1/1 [==============================] - 0s 12ms/step - loss: 2.1481
Epoch 5/500
1/1 [==============================] - 0s 17ms/step - loss: 1.8425
Epoch 6/500
1/1 [==============================] - 0s 10ms/step - loss: 1.5989
Epoch 7/500
1/1 [==============================] - 0s 9ms/step - loss: 1.4042
Epoch 8/500
1/1 [==============================] - 0s 15ms/step - loss: 1.2481
Epoch 9/500
1/1 [==============================] - 0s 14ms/step - loss: 1.1222
Epoch 10/500
1/1 [==============================] - 0s 11ms/step - loss: 1.0204
Epoch 11/500
1/1 [==============================] - 0s 8ms/step - loss: 0.9374
Epoch 12/500
1/1 [==============================] - 0s 8ms/step - loss: 0.8694
Epoch 13/500
1/1 [==============================] - 

1092

In [ ]:
######################################
# 1-2. 기본: tflite 생성 후, 결과 확인
####################################

import tensorflow as tf
import tflite_runtime.interpreter as tflite
import numpy as np

#모델 로드
interpreter = tf.lite.Interpreter(model_path='/content/drive/MyDrive/INTEL_PYTHON/model.tflite')
#텐서 할당
interpreter.allocate_tensors()

#모델이 기대하는 입력 데이터 포맷
input_details = interpreter.get_input_details()
#모델이 기대하는 출력 데이터 포맷
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

#입력 데이터(x=10.0)에 대한 y를 예측하려면 입력 배열의 크기와 타입 정의  필요
to_predict =  np.array([[10.0]],dtype = np.float32)
print(to_predict)

#입력 값을 넣기 위한 입력 텐서 설정
interpreter.set_tensor(input_details[0]['index'], to_predict)
interpreter.invoke()

#출력 값을 반환 받을 출력 텐서 설정
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(tflite_results)

[{'name': 'serving_default_dense_input:0', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 3, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[[10.]]
[[18.978815]]


In [ ]:
###############################
# 2-2. 전이학습으로 모델 구현 및 metadata 확인
################################

import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

#244x 244 사이즈로 리사이즈를 하고, 정규화를 해줍니다.
def format_image(image,label):
    image = tf.image.resize(image,(224,224))/255.0
    return image, label

#훈련, 검증, 테스트 세트로 나눕니다.
(raw_train, raw_validation, raw_test), metadata =tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]','train[80%:90%]','train[90%:]'],
    with_info=True,
    as_supervised=True,
)

print(metadata)

num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes



tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    full_name='cats_vs_dogs/4.0.1',
    description="""
    A large set of images of cats and dogs. There are 1738 corrupted images that are dropped.
    """,
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    data_dir='/root/tensorflow_datasets/cats_vs_dogs/4.0.1',
    file_format=tfrecord,
    download_size=786.67 MiB,
    dataset_size=1.04 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=23262, num_shards=16>,
    },
    citation="""@Inproceedings (Conference){asirra-a-captcha-that-exploits-interest-aligned-manual-image-categorization,
    author = {Elson, Jeremy and Douceur, John (JD) and Howell, Jon and Saul, 

In [ ]:
###############################
# 2-1. 전이학습으로 모델 구현 + tflite로 변환
################################

import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

#244x 244 사이즈로 리사이즈를 하고, 정규화를 해줍니다.
def format_image(image,label):
    image = tf.image.resize(image,(224,224))/255.0
    return image, label

#훈련, 검증, 테스트 세트로 나눕니다.
(raw_train, raw_validation, raw_test), metadata =tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]','train[80%:90%]','train[90%:]'],
    with_info=True,
    as_supervised=True,
)


num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes

BATCH_SIZE = 32
train_batches = raw_train.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)

validation_batches = raw_validation.map(format_image).batch(BATCH_SIZE).prefetch(1)

test_batches = raw_test.map(format_image).batch(1)

#텐서플로우 허브에 있는 mobilenet_v2 모델을 사용해 feature_extractor라는 케라스 층을 만듭니다.
#신경망의 첫번째 층
handle_base, pixels, FV_SIZE = ("mobilenet_v2", 224, 1280)

MODULE_HANDLE = "https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)

IMAGE_SIZE =(pixels, pixels)


feature_extractor = hub.KerasLayer(
    MODULE_HANDLE,
    input_shape = IMAGE_SIZE +(3,),
    output_shape = [FV_SIZE],
    trainable = False
)

# 모델 선언
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(num_classes, activation ='softmax')
])

model.compile(
    optimizer='adam',
    loss ='sparse_categorical_crossentropy',
    metrics =['accuracy']
)

hist = model.fit(
    train_batches,
    epochs = 5,
    validation_data = validation_batches
)

#학습된 모델 저장하기
#colab 버전
CATS_VS_DOGS_SAVED_MODEL = "/content/drive/MyDrive/INTEL_PYTHON/exp_saved_model"
#pc 버전
#CATS_VS_DOGS_SAVED_MODEL = 'exp_saved_model'
tf.saved_model.save(model, CATS_VS_DOGS_SAVED_MODEL)


#tensoflow lite로 변환하기
converter = tf.lite.TFLiteConverter.from_saved_model(CATS_VS_DOGS_SAVED_MODEL)
tflite_model = converter.convert()
tflite_model_file = '/content/drive/MyDrive/INTEL_PYTHON/converted_model.tflite'
with open(tflite_model_file, "wb") as f:
  f.write(tflite_model)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.1.incompleteMIBAI8/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.
Epoch 1/5
582/582 [==============================] - 51s 65ms/step - loss: 0.0535 - accuracy: 0.9801 - val_loss: 0.0422 - val_accuracy: 0.9850
Epoch 2/5
582/582 [==============================] - 41s 61ms/step - loss: 0.0306 - accuracy: 0.9898 - val_loss: 0.0395 - val_accuracy: 0.9862
Epoch 3/5
582/582 [==============================] - 39s 56ms/step - loss: 0.0248 - accuracy: 0.9913 - val_loss: 0.0393 - val_accuracy: 0.9884
Epoch 4/5
582/582 [==============================] - 42s 61ms/step - loss: 0.0215 - accuracy: 0.9924 - val_loss: 0.0387 - val_accuracy: 0.9867
Epoch 5/5
582/582 [==============================] - 40s 58ms/step - loss: 0.0191 - accuracy: 0.9938 - val_loss: 0.0410 - val_accuracy: 0.9875


In [ ]:
###############################
# 2-2. 전이학습+최적화(동적 범위 양자화 (dynamic range quantization))
# + tflite로 변환
################################

import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

#244x 244 사이즈로 리사이즈를 하고, 정규화를 해줍니다.
def format_image(image,label):
    image = tf.image.resize(image,(224,224))/255.0
    return image, label

#훈련, 검증, 테스트 세트로 나눕니다.
(raw_train, raw_validation, raw_test), metadata =tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]','train[80%:90%]','train[90%:]'],
    with_info=True,
    as_supervised=True,
)


num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes

BATCH_SIZE = 32
train_batches = raw_train.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)

validation_batches = raw_validation.map(format_image).batch(BATCH_SIZE).prefetch(1)

test_batches = raw_test.map(format_image).batch(1)

#텐서플로우 허브에 있는 mobilenet_v2 모델을 사용해 feature_extractor라는 케라스 층을 만듭니다.
#신경망의 첫번째 층
handle_base, pixels, FV_SIZE = ("mobilenet_v2", 224, 1280)

MODULE_HANDLE = "https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)

IMAGE_SIZE =(pixels, pixels)


feature_extractor = hub.KerasLayer(
    MODULE_HANDLE,
    input_shape = IMAGE_SIZE +(3,),
    output_shape = [FV_SIZE],
    trainable = False
)

# 모델 선언
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(num_classes, activation ='softmax')
])

model.compile(
    optimizer='adam',
    loss ='sparse_categorical_crossentropy',
    metrics =['accuracy']
)

hist = model.fit(
    train_batches,
    epochs = 5,
    validation_data = validation_batches
)

#학습된 모델 저장하기
#colab 버전
CATS_VS_DOGS_SAVED_MODEL = '/content/drive/MyDrive/INTEL_PYTHON/drq_exp_saved_model'
tf.saved_model.save(model, CATS_VS_DOGS_SAVED_MODEL)


#tensoflow lite로 변환하기
converter = tf.lite.TFLiteConverter.from_saved_model(CATS_VS_DOGS_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
tflite_model_file = '/content/drive/MyDrive/INTEL_PYTHON/drq_converted_model.tflite'
with open(tflite_model_file, "wb") as f:
  f.write(tflite_model)


interpreter = tf.lite.Interpreter(model_content=tflite_model)
input_details = interpreter.get_input_details()[0]
print('input: ', input_details)
output_details = interpreter.get_output_details()[0]
print('output: ', output_details)


Epoch 1/5
582/582 [==============================] - 74s 79ms/step - loss: 0.0507 - accuracy: 0.9833 - val_loss: 0.0400 - val_accuracy: 0.9875
Epoch 2/5
582/582 [==============================] - 51s 73ms/step - loss: 0.0305 - accuracy: 0.9895 - val_loss: 0.0379 - val_accuracy: 0.9875
Epoch 3/5
582/582 [==============================] - 43s 64ms/step - loss: 0.0249 - accuracy: 0.9917 - val_loss: 0.0438 - val_accuracy: 0.9867
Epoch 4/5
582/582 [==============================] - 45s 66ms/step - loss: 0.0215 - accuracy: 0.9931 - val_loss: 0.0427 - val_accuracy: 0.9875
Epoch 5/5
582/582 [==============================] - 46s 65ms/step - loss: 0.0180 - accuracy: 0.9943 - val_loss: 0.0426 - val_accuracy: 0.9867
input:  {'name': 'serving_default_keras_layer_input:0', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([

In [ ]:
###############################
# 2-2.전이학습+최적화(DRQ) + tflite로 변환
# (추가) tf.lite.experimental.Analyzer.analyze 결과 보기
################################

import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

#244x 244 사이즈로 리사이즈를 하고, 정규화를 해줍니다.
def format_image(image,label):
    image = tf.image.resize(image,(224,224))/255.0
    return image, label

#훈련, 검증, 테스트 세트로 나눕니다.
(raw_train, raw_validation, raw_test), metadata =tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]','train[80%:90%]','train[90%:]'],
    with_info=True,
    as_supervised=True,
)


num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes

BATCH_SIZE = 32
train_batches = raw_train.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)

validation_batches = raw_validation.map(format_image).batch(BATCH_SIZE).prefetch(1)

test_batches = raw_test.map(format_image).batch(1)

#텐서플로우 허브에 있는 mobilenet_v2 모델을 사용해 feature_extractor라는 케라스 층을 만듭니다.
#신경망의 첫번째 층
handle_base, pixels, FV_SIZE = ("mobilenet_v2", 224, 1280)

MODULE_HANDLE = "https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)

IMAGE_SIZE =(pixels, pixels)


feature_extractor = hub.KerasLayer(
    MODULE_HANDLE,
    input_shape = IMAGE_SIZE +(3,),
    output_shape = [FV_SIZE],
    trainable = False
)

# 모델 선언
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(num_classes, activation ='softmax')
])

model.summary()

model.compile(
    optimizer='adam',
    loss ='sparse_categorical_crossentropy',
    metrics =['accuracy']
)

hist = model.fit(
    train_batches,
    epochs = 5,
    validation_data = validation_batches
)


#학습된 모델 저장하기
CATS_VS_DOGS_SAVED_MODEL = '/content/drive/MyDrive/INTEL_PYTHON/exp_saved_model'
tf.saved_model.save(model, CATS_VS_DOGS_SAVED_MODEL)


#tensoflow lite로 변환하기
converter = tf.lite.TFLiteConverter.from_saved_model(CATS_VS_DOGS_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)

tflite_model_file = '/content/drive/MyDrive/INTEL_PYTHON/drq_converted_model.tflite'
with open(tflite_model_file, "wb") as f:
  f.write(tflite_model)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense_1 (Dense)             (None, 2)                 2562      
                                                                 
Total params: 2260546 (8.62 MB)
Trainable params: 2562 (10.01 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________
Epoch 1/5
582/582 [==============================] - 53s 69ms/step - loss: 0.0523 - accuracy: 0.9822 - val_loss: 0.0420 - val_accuracy: 0.9850
Epoch 2/5
582/582 [==============================] - 44s 60ms/step - loss: 0.0298 - accuracy: 0.9896 - val_loss: 0.0391 - val_accuracy: 0.9880
Epoch 3/5
582/582 [==============================] - 45s 67ms/step - loss: 0.0253 - accuracy: 0.9911 - val_loss: 0.0384 - va

In [ ]:
###############################
# 2-3.전이학습+최적화(float16 quantization) + tflite로 변환
################################

import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

#244x 244 사이즈로 리사이즈를 하고, 정규화를 해줍니다.
def format_image(image,label):
    image = tf.image.resize(image,(224,224))/255.0
    return image, label

#훈련, 검증, 테스트 세트로 나눕니다.
(raw_train, raw_validation, raw_test), metadata =tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]','train[80%:90%]','train[90%:]'],
    with_info=True,
    as_supervised=True,
)


num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes

BATCH_SIZE = 32
train_batches = raw_train.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)

validation_batches = raw_validation.map(format_image).batch(BATCH_SIZE).prefetch(1)

test_batches = raw_test.map(format_image).batch(1)

#텐서플로우 허브에 있는 mobilenet_v2 모델을 사용해 feature_extractor라는 케라스 층을 만듭니다.
#신경망의 첫번째 층
handle_base, pixels, FV_SIZE = ("mobilenet_v2", 224, 1280)

MODULE_HANDLE = "https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)

IMAGE_SIZE =(pixels, pixels)


feature_extractor = hub.KerasLayer(
    MODULE_HANDLE, #모델
    input_shape = IMAGE_SIZE +(3,), #(224,224,3) # Outputs a tensor with shape [batch_size, 20].
    output_shape = [FV_SIZE], # Expects a tensor of shape [batch_size] as input.
    trainable = False
)

# 모델 선언
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(num_classes, activation ='softmax')
])

model.compile(
    optimizer='adam',
    loss ='sparse_categorical_crossentropy',
    metrics =['accuracy']
)

hist = model.fit(
    train_batches,
    epochs = 5,
    validation_data = validation_batches
)

model.summary()

#학습된 모델 저장하기

CATS_VS_DOGS_SAVED_MODEL = '/content/drive/MyDrive/INTEL_PYTHON/exp_saved_model'
tf.saved_model.save(model, CATS_VS_DOGS_SAVED_MODEL)


converter = tf.lite.TFLiteConverter.from_saved_model(CATS_VS_DOGS_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types=[tf.float16]

tflite_model_ffq = converter.convert()
tflite_model_file = '/content/drive/MyDrive/INTEL_PYTHON/f16q_converted_model.tflite'
with open(tflite_model_file, "wb") as f:
  f.write(tflite_model_ffq)


Epoch 1/5
582/582 [==============================] - 45s 56ms/step - loss: 0.0624 - accuracy: 0.9773 - val_loss: 0.0398 - val_accuracy: 0.9854
Epoch 2/5
582/582 [==============================] - 37s 53ms/step - loss: 0.0313 - accuracy: 0.9892 - val_loss: 0.0351 - val_accuracy: 0.9905
Epoch 3/5
582/582 [==============================] - 37s 53ms/step - loss: 0.0265 - accuracy: 0.9912 - val_loss: 0.0387 - val_accuracy: 0.9867
Epoch 4/5
582/582 [==============================] - 36s 52ms/step - loss: 0.0232 - accuracy: 0.9922 - val_loss: 0.0379 - val_accuracy: 0.9884
Epoch 5/5
582/582 [==============================] - 37s 55ms/step - loss: 0.0192 - accuracy: 0.9932 - val_loss: 0.0383 - val_accuracy: 0.9880
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 

In [ ]:
###############################
# 케라스 허브에서 모델 가져오기
# https://www.kaggle.com/
# mobilenet 검색 (_ v2)
################################

import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

#244x 244 사이즈로 리사이즈를 하고, 정규화를 해줍니다.
def format_image(image,label):
    image = tf.image.resize(image,(224,224))/255.0
    return image, label

#훈련, 검증, 테스트 세트로 나눕니다.
(raw_train, raw_validation, raw_test), metadata =tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]','train[80%:90%]','train[90%:]'],
    with_info=True,
    as_supervised=True,
)

print(metadata)

num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes


BATCH_SIZE = 32
train_batches = raw_train.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)

validation_batches = raw_validation.map(format_image).batch(BATCH_SIZE).prefetch(1)

test_batches = raw_test.map(format_image).batch(1)


####################
#케라스 허브에서 모델 가져오기
###################
model = tf.keras.Sequential([
    hub.KerasLayer("https://www.kaggle.com/models/google/mobilenet-v2/frameworks/TensorFlow2/variations/tf2-preview-feature-vector/versions/4",
                   input_shape = (224,224,3),
                   output_shape=[1280],
                   trainable=False),  # Can be True, see below.
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss ='sparse_categorical_crossentropy',
    metrics =['accuracy']
)

hist = model.fit(
    train_batches,
    epochs = 5,
    validation_data = validation_batches
)

model.summary()

#학습된 모델 저장하기
CATS_VS_DOGS_SAVED_MODEL = '/content/drive/MyDrive/INTEL_PYTHON/exp2_saved_model'
tf.saved_model.save(model, CATS_VS_DOGS_SAVED_MODEL)

converter = tf.lite.TFLiteConverter.from_saved_model(CATS_VS_DOGS_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types=[tf.float16]

tflite_model_ffq = converter.convert()
tflite_model_file = '/content/drive/MyDrive/INTEL_PYTHON/f16q_converted_model_v2.tflite'
with open(tflite_model_file, "wb") as f:
  f.write(tflite_model_ffq)



Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.1.incompleteIUS0II/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    full_name='cats_vs_dogs/4.0.1',
    description="""
    A large set of images of cats and dogs. There are 1738 corrupted images that are dropped.
    """,
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    data_dir=PosixGPath('/tmp/tmpsidmo9vptfds'),
    file_format=tfrecord,
    download_size=786.67 MiB,
    dataset_size=1.04 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=23262, num_shards=16>,
    },
    citation="""@Inproceedings (Conference){asirra-a-captcha-that-exploits-in

In [ ]:
###############################
# 2-4.전이학습+최적화(float fallback quantization)
# + tflite로 변환
# weights and variable data are quantized
################################
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

#244x 244 사이즈로 리사이즈를 하고, 정규화를 해줍니다.
def format_image(image,label):
    image = tf.image.resize(image,(224,224))/255.0
    return image, label

#훈련, 검증, 테스트 세트로 나눕니다.
(raw_train, raw_validation, raw_test), metadata =tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]','train[80%:90%]','train[90%:]'],
    with_info=True,
    as_supervised=True,
)

print(metadata)

num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes


BATCH_SIZE = 32
train_batches = raw_train.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)

validation_batches = raw_validation.map(format_image).batch(BATCH_SIZE).prefetch(1)

test_batches = raw_test.map(format_image).batch(1)

#텐서플로우 허브에 있는 mobilenet_v2 모델을 사용해 feature_extractor라는 케라스 층을 만듭니다.
#신경망의 첫번째 층
handle_base, pixels, FV_SIZE = ("mobilenet_v2", 224, 1280)

MODULE_HANDLE = "https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)

IMAGE_SIZE =(pixels, pixels)


feature_extractor = hub.KerasLayer(
    MODULE_HANDLE, #모델
    input_shape = IMAGE_SIZE +(3,),# #(224,224,3) Expects a tensor of shape [batch_size] as input.
    output_shape = [FV_SIZE], # Outputs a tensor with shape [batch_size, 20].
    trainable = False
)

# 모델 선언
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(num_classes, activation ='softmax')
])

model.compile(
    optimizer='adam',
    loss ='sparse_categorical_crossentropy',
    metrics =['accuracy']
)

hist = model.fit(
    train_batches,
    epochs = 5,
    validation_data = validation_batches
)

model.summary()

#학습된 모델 저장하기
CATS_VS_DOGS_SAVED_MODEL = '/content/drive/MyDrive/INTEL_PYTHON/exp_saved_model'
tf.saved_model.save(model, CATS_VS_DOGS_SAVED_MODEL)

#########################
# 대표 샘플을 뽑아서, variable data 양자화
############################
def representative_data_gen():
  for input_value, _ in test_batches.take(100):
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_saved_model(CATS_VS_DOGS_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

tflite_model_ffq = converter.convert()
tflite_model_file = '/content/drive/MyDrive/INTEL_PYTHON/ffq_converted_model.tflite'
with open(tflite_model_file, "wb") as f:
  f.write(tflite_model_ffq)


interpreter = tf.lite.Interpreter(model_content=tflite_model_ffq)
input_details = interpreter.get_input_details()[0]
print('input: ', input_details)
output_details = interpreter.get_output_details()[0]
print('output: ', output_details)


tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    full_name='cats_vs_dogs/4.0.1',
    description="""
    A large set of images of cats and dogs. There are 1738 corrupted images that are dropped.
    """,
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    data_dir='/root/tensorflow_datasets/cats_vs_dogs/4.0.1',
    file_format=tfrecord,
    download_size=786.67 MiB,
    dataset_size=1.04 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=23262, num_shards=16>,
    },
    citation="""@Inproceedings (Conference){asirra-a-captcha-that-exploits-interest-aligned-manual-image-categorization,
    author = {Elson, Jeremy and Douceur, John (JD) and Howell, Jon and Saul, 

In [2]:
###############################
# 2-4.전이학습+최적화(완전한 정수 양자화 (full integer quantization))
# + tflite로 변환
# weights and variable data are quantized
################################

import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

#244x 244 사이즈로 리사이즈를 하고, 정규화를 해줍니다.
def format_image(image,label):
    image = tf.image.resize(image,(224,224))/255.0
    return image, label

#훈련, 검증, 테스트 세트로 나눕니다.
(raw_train, raw_validation, raw_test), metadata =tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]','train[80%:90%]','train[90%:]'],
    with_info=True,
    as_supervised=True,
)

print(metadata)

num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes


BATCH_SIZE = 32
train_batches = raw_train.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)

validation_batches = raw_validation.map(format_image).batch(BATCH_SIZE).prefetch(1)

test_batches = raw_test.map(format_image).batch(1)

#텐서플로우 허브에 있는 mobilenet_v2 모델을 사용해 feature_extractor라는 케라스 층을 만듭니다.
#신경망의 첫번째 층
handle_base, pixels, FV_SIZE = ("mobilenet_v2", 224, 1280)

MODULE_HANDLE = "https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)

IMAGE_SIZE =(pixels, pixels)


feature_extractor = hub.KerasLayer(
    MODULE_HANDLE, #모델
    input_shape = IMAGE_SIZE +(3,),# #(224,224,3) Expects a tensor of shape [batch_size] as input.
    output_shape = [FV_SIZE], # Outputs a tensor with shape [batch_size, 20].
    trainable = False
)

# 모델 선언
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(num_classes, activation ='softmax')
])

model.compile(
    optimizer='adam',
    loss ='sparse_categorical_crossentropy',
    metrics =['accuracy']
)

hist = model.fit(
    train_batches,
    epochs = 5,
    validation_data = validation_batches
)

model.summary()

#학습된 모델 저장하기
CATS_VS_DOGS_SAVED_MODEL = '/content/drive/MyDrive/INTEL_PYTHON/exp_saved_model'
tf.saved_model.save(model, CATS_VS_DOGS_SAVED_MODEL)

#tensoflow lite로 변환하기
def representative_data_gen():
  for input_value, _ in test_batches.take(100):
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_saved_model(CATS_VS_DOGS_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_fiq = converter.convert()
tflite_model_file = '/content/drive/MyDrive/INTEL_PYTHON/fiq_converted_model.tflite'
with open(tflite_model_file, "wb") as f:
  f.write(tflite_model_fiq)

interpreter = tf.lite.Interpreter(model_content=tflite_model_fiq)
input_details = interpreter.get_input_details()[0]
print('input: ', input_details)
output_details = interpreter.get_output_details()[0]
print('output: ', output_details)


tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    full_name='cats_vs_dogs/4.0.1',
    description="""
    A large set of images of cats and dogs. There are 1738 corrupted images that are dropped.
    """,
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    data_dir='/root/tensorflow_datasets/cats_vs_dogs/4.0.1',
    file_format=tfrecord,
    download_size=786.67 MiB,
    dataset_size=1.04 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=23262, num_shards=16>,
    },
    citation="""@Inproceedings (Conference){asirra-a-captcha-that-exploits-interest-aligned-manual-image-categorization,
    author = {Elson, Jeremy and Douceur, John (JD) and Howell, Jon and Saul, 